In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import super_res
from malaya_speech.train.model import enhancement
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 8192)
model = super_res.UNET(partitioned_x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Tensor("generator/downsc_conv0/leaky_re_lu/LeakyRelu:0", shape=(?, 4096, 128), dtype=float32)
Tensor("generator/downsc_conv1/leaky_re_lu_1/LeakyRelu:0", shape=(?, 2048, 384), dtype=float32)
Tensor("generator/downsc_conv2/leaky_re_lu_2/LeakyRelu:0", shape=(?, 1024, 512), dtype=float32)
Tensor("generator/downsc_conv3/leaky_re_lu_3/LeakyRelu:0", shape=(?, 512, 512), dtype=float32)
Tensor("generator/bottleneck_conv/leaky_re_lu_4/LeakyRelu:0", shape=(?, 256, 512), dtype=float32)

Tensor("generator/upsc_conv3/concat:0", shape=(?, 512, 1024), dtype=float32)
Tensor("generator/upsc_conv2/concat:0", shape=(?, 1024, 1024), dtype=float32)
Tensor("generator/upsc_conv1/concat:0", shape=(?, 2048, 768), dtype=float32)
Tensor("generator/upsc_conv0/concat:0", shape=(?, 4096, 256), dtype=float32)
Tensor("generator/lastconv/subpixel/transpose_1:0", shape=(?, 8192, 1), dtype=float32)


In [5]:
model.logits

<tf.Tensor 'generator/add:0' shape=(?, 8192, 1) dtype=float32>

In [6]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(x)[0]]
logits

<tf.Tensor 'strided_slice_4:0' shape=(?, 1) dtype=float32>

In [7]:
snr = enhancement.loss.snr(model.logits, partitioned_x)
sdr = enhancement.loss.sdr(model.logits, partitioned_x)

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [9]:
y, sr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 22050)

In [21]:
%%time

y_ = sess.run(model.logits, feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

CPU times: user 1min 59s, sys: 1.5 s, total: 2min
Wall time: 16.4 s


(32, 8192, 1)

In [11]:
y_.reshape((-1)).shape

(262144,)

In [12]:
y.shape

(258867,)

In [13]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [14]:
y_, snr_, sdr_ = sess.run([model.logits, snr, sdr], feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(32, 8192, 1)

In [15]:
snr_, sdr_

((0.0073915836, 19.589632), -3.7994573e-06)

In [16]:
saver = tf.train.Saver()

In [17]:
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [18]:
!ls -lh test

total 554752
-rw-r--r--  1 huseinzolkepli  staff    77B Jan 12 17:38 checkpoint
-rw-r--r--  1 huseinzolkepli  staff   271M Jan 12 17:38 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff   993B Jan 12 17:38 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   108K Jan 12 17:38 model.ckpt.meta


In [19]:
tf.trainable_variables()

[<tf.Variable 'generator/downsc_conv0/conv1d/kernel:0' shape=(65, 1, 128) dtype=float32>,
 <tf.Variable 'generator/downsc_conv0/conv1d/bias:0' shape=(128,) dtype=float32>,
 <tf.Variable 'generator/downsc_conv1/conv1d_1/kernel:0' shape=(33, 128, 384) dtype=float32>,
 <tf.Variable 'generator/downsc_conv1/conv1d_1/bias:0' shape=(384,) dtype=float32>,
 <tf.Variable 'generator/downsc_conv2/conv1d_2/kernel:0' shape=(17, 384, 512) dtype=float32>,
 <tf.Variable 'generator/downsc_conv2/conv1d_2/bias:0' shape=(512,) dtype=float32>,
 <tf.Variable 'generator/downsc_conv3/conv1d_3/kernel:0' shape=(9, 512, 512) dtype=float32>,
 <tf.Variable 'generator/downsc_conv3/conv1d_3/bias:0' shape=(512,) dtype=float32>,
 <tf.Variable 'generator/bottleneck_conv/conv1d_4/kernel:0' shape=(9, 512, 512) dtype=float32>,
 <tf.Variable 'generator/bottleneck_conv/conv1d_4/bias:0' shape=(512,) dtype=float32>,
 <tf.Variable 'generator/upsc_conv3/conv1d_5/kernel:0' shape=(9, 512, 1024) dtype=float32>,
 <tf.Variable 'gener

In [20]:
!rm -rf test